# Solution Optimization Evaluaton TV3 TextGrad

In [1]:
import pandas as pd
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierV4
from textgrad.loss import TextLoss

## Load Datasets

In [2]:
initial_solution = pd.read_csv("csv/initial_solution.csv")
initial_solution

,id,formatted_question,raw_solution,correct_answer,source,subject
0,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-
1,4,Answer the following multiple choice question....,Maxwell's equations in our universe are:\n\n1....,A,GPQA-Diamond,-
2,8,Answer the following multiple choice question....,Here's how we can analyze the results and dete...,B,GPQA-Diamond,-
3,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,A,GPQA-Diamond,-
4,22,Answer the following multiple choice question....,The question asks about the oxidizing power of...,D,GPQA-Diamond,-
...,...,...,...,...,...,...
407,394,Answer the following multiple choice question....,The police car is moving towards the wall. Le...,B,MMLU-CP,college_physics
408,384,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,A,MMLU-CP,college_physics
409,404,Answer the following multiple choice question....,The diffraction of electrons by a crystal latt...,A,MMLU-CP,college_physics
410,390,Answer the following multiple choice question....,Here's how we can solve this problem:\n\n1. **...,D,MMLU-CP,college_physics


In [3]:
# Test size only 50 rows each datasets (Total 150 rows)

df_gpqa = initial_solution[initial_solution['source'] == 'GPQA-Diamond'].head(50)
df_mmlu_ml = initial_solution[initial_solution['source'] == 'MMLU-ML'].head(50)
df_mmlu_cp = initial_solution[initial_solution['source'] == 'MMLU-CP'].head(50)
df_test = pd.concat([df_gpqa, df_mmlu_ml, df_mmlu_cp], ignore_index=True)

df_test

,id,formatted_question,raw_solution,correct_answer,source,subject
0,2,Answer the following multiple choice question....,Here's how we can determine the number of carb...,A,GPQA-Diamond,-
1,4,Answer the following multiple choice question....,Maxwell's equations in our universe are:\n\n1....,A,GPQA-Diamond,-
2,8,Answer the following multiple choice question....,Here's how we can analyze the results and dete...,B,GPQA-Diamond,-
3,1,Answer the following multiple choice question....,The energy-time uncertainty principle states t...,A,GPQA-Diamond,-
4,22,Answer the following multiple choice question....,The question asks about the oxidizing power of...,D,GPQA-Diamond,-
...,...,...,...,...,...,...
145,339,Answer the following multiple choice question....,The proton is initially accelerated through a ...,D,MMLU-CP,college_physics
146,388,Answer the following multiple choice question....,Einstein's theory of the photoelectric effect ...,D,MMLU-CP,college_physics
147,364,Answer the following multiple choice question....,We are given that the mass of object B is twic...,C,MMLU-CP,college_physics
148,380,Answer the following multiple choice question....,"The electric displacement current, denoted by ...",A,MMLU-CP,college_physics


## Experiment

In [4]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def evaluate_with_raw_textgrad(row_data):
    match = initial_solution[initial_solution["id"] == row_data["id"]]
    if match.empty:
        return None  # or raise error
    formatted_question = match.iloc[0]["formatted_question"]
    result = {
        "id": row_data["id"],
        "raw_solution": row_data["raw_solution"],
        "correct_answer": row_data["correct_answer"],
        "source": row_data["source"],
        "subject": row_data["subject"]
    }
    
    solution = Variable(row_data["raw_solution"],
                    requires_grad=True,
                    role_description=f"Solution to the math question: {formatted_question}")
    loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                    Do not attempt to solve it yourself, do not give a solution, 
                                    only identify errors. Be super concise.""",
                                    requires_grad=False,
                                    role_description="system prompt")
    optimizer = TextualGradientDescent([solution])
    loss = TextLoss(loss_system_prompt, engine=engine)

    # TextualVerifierV4
    verifier = TextualVerifierV4(verifier_engine=engine, step_eval_iterations=3, logger=False)
    
    # Iterate 5 times
    for i in range(1, 6):
        optimizer.zero_grad()  # Clean gradients
        loss_result = loss(solution)

        # TextualVerifierV4
        verified_result = verifier.verify(instance=solution, 
                                    prompt=loss_system_prompt,
                                    calculation=loss_result)
        loss_result.set_value(verified_result.value) 
        
        loss_result.backward()
        optimizer.step()
        result[f"solution_{i}"] = solution.value

    return result

## Running Evaluation

### TV TextGrad

In [6]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

results = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=128) as executor:
    # Submit all tasks
    futures = [
        executor.submit(evaluate_with_raw_textgrad, row.to_dict()) 
        for _, row in initial_solution[:10].iterrows()
    ]
    
    # Use tqdm for progress tracking
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
        result = future.result()
        if result is not None:
            results.append(result)

raw_textgrad = pd.DataFrame(results)

print(f"Completed in {time.time() - start_time:.1f} seconds")
raw_textgrad.to_csv('results/tv4_textgrad.csv', index=False)

Processing:   0%|          | 0/10 [00:00<?, ?it/s]

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The energy of the emitted light is given as 2.3393 eV. We need to find the color of light absorbed by the compound.  The absorbed light is complementary to the emitted light.  First, we need to convert the energy to wavelength.\n\nThe energy-wavelength relationship is given by:\n\nE = h * c / λ\n\nwhere:\nE is the energy in Joules\nh is Planck\'s constant (6.626 x 10^-34 J s)\nc is the speed of light (3 x 10^8 m/s)\nλ is the wavelength in meters\n\nFirst, convert the energy from eV to Joules:\n\nE = 2.3393 eV * (1.602 x 10^-19 J/eV) = 3.748 x 10^-19 J\n\nNow, we can calculate the wavelength:\n\nλ = h * c / E\nλ = (6.626 x 10^-34 J s) * (3 x 10^8 m/s) / (3.748 x 10^-19 J

Processing:  10%|█         | 1/10 [00:05<00:51,  5.69s/it]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> * The reaction is a Diels-Alder reaction, a [4+2] cycloaddition.\n* 2,5-dimethylthiophene acts as the diene.\n* Furan-2,5-dione (maleic anhydride) acts as the dienophile.\n* The reaction proceeds with heat.\n* The product will be a bicyclic structure.\n* The two methyl groups on the thiophene will end up on the bridgehead carbons of the bicyclic product.\n* Since the methyl groups are at positions 2 and 5 on the thiophene, they will be on the same side of the newly formed six-membered ring in the product. This means they will have a *cis* relationship.\n* The oxygen bridge from the maleic anhydride will be *syn* to the sulfur in the thiophene ring.  This leads to the *e

Processing:  20%|██        | 2/10 [00:07<00:26,  3.37s/it]

...

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> The energy-time uncertainty principle states that ΔE * Δt >= h-bar/2. To distinguish two energy levels, their difference must be greater than the uncertainty associated with *each* energy level individually. This ensures both levels are clearly resolved.\n\nGiven lifetimes (Δt) of 10^-9 s and 10^-8 s, and h-bar ≈ 6.58 * 10^-16 eV*s:\n\nΔE1 = h-bar / (2 * 10^-9 s) ≈ 3.29 * 10^-7 eV\nΔE2 = h-bar / (2 * 10^-8 s) ≈ 3.29 * 10^-8 eV\n\nThe uncertainties are ΔE1 ≈ 3.29 * 10^-7 eV and ΔE2 ≈ 3.29 * 10^-8 eV.  To resolve both levels, the energy difference must be greater than the *larger* of these two uncertainties, which is ΔE1.\n\nComparing this to the options:\nA) 10^-4 eV > 3

Processing:  90%|█████████ | 9/10 [02:45<00:15, 15.73s/it]

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \n                                    Do not attempt to solve it yourself, do not give a solution, \n                                    only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> Here's how we can analyze the results:\n\n1. **G2's Importance:** The g2 mutant (and any double mutant including g2) shows 0% resistance. This strongly suggests G2 is essential for resistance and the most likely candidate for the transcription factor role.\n\n2. **G1 and G3's Roles:** Individually, g1 and g3 mutants show some resistance (75% and 50%, respectively). This suggests they contribute to the overall resistance response, potentially acting downstream of G2.\n\n3. **Interaction between G1 and G3:** The g1g3 double mutant shows drastically reduced resistance (10%) compared to either g1 or g3 alone. This synergistic effect indicates that G1 and G3 likely work toge

Processing: 100%|██████████| 10/10 [03:49<00:00, 22.99s/it]

Completed in 230.0 seconds
